In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller

In [ ]:
dollar_df = pd.read_csv('../data/processed_sp500_futures_dollar.csv')

In [ ]:
dollar_df.head()

#### Apply the Brown-Durbin-Evans method

In [ ]:
def solve_lse(Xt, yt):
    return np.linalg.lstsq(Xt, yt, rcond=None)

In [ ]:
def pick_k(X, y):
    return len(y) * 999 // 1000

In [ ]:
X = dollar_df[['dollar_group', 'total_volume', 'count']]
X = (X - X.mean()) / X.std()
y = dollar_df.close.values.reshape(-1, 1)

In [ ]:
k = pick_k(X, y)
T = len(y)

# b values computed from data up to index t (key) inclusive
b_values = {} # keys: k -> T-1
count = 0
for t in range(k, T):
    count += 1
    if count >= (T-k) / 10:
        print("10% Completed")
        count = 0

    b_values[t] = solve_lse(X.iloc[:t+1, :], y[:t+1]) # t+1 first values

In [ ]:
# Temporarily set this value to 1, following the formula from the original paper
# https://hhstokes.people.uic.edu/ftp/e535/Brown_Durbin_evans_1975.pdf
noise_var = 1
weights = {} # keys: from k up to T-1

# Loop from t: k-1 -> T-2
for t in range(k, T):
    if (t-1) not in b_values.keys():
        continue

    f_t = 1 + X.iloc[t,:].T @ np.linalg.inv(X.iloc[:t+1].T @ X.iloc[:t+1]) @ X.iloc[t,:]    
    w_t = (y[t] - b_values[t-1][0].reshape(1, -1).T @ X.iloc[t,:].to_numpy().reshape(1, -1)) / np.sqrt(f_t)
    weights[t] = w_t

In [ ]:
S_t = {}
weight_std = np.std(list(weights.values()))
for t in range(min(weights.keys()), max(weights.keys()) + 1):
    weight_cumsum = 0
    for i in range(k, t+1):
        weight_cumsum += weights.get(i, 0)#
    
    S_t[t] = weight_cumsum / weight_std

#### Apply the Chu-Stinchcombe-White method

TBD

#### Chow-Type Dickey Fuller

In [ ]:
def dickey_fuller_chow_stat(ts, cutoff):
    None

#### Supremum Augmented Dickey-Fuller

In [ ]:
def compute_sdaf(timeseries, min_sample_ratio=0.5/100):
    sdaf = {}
    N = timeseries.shape[0]
    min_samples = int(N * 0.5 / 100)
    for t in range(min_samples, N):
        min_adf = float('inf')
        for t0 in range(t-min_samples+1):
            adf_result = adfuller(timeseries[t0:t+1])
            min_adf = min(min_adf, adf_result[0])
        sdaf[t] = min_adf
    return sdaf

In [ ]:
sdaf = compute_sdaf(dollar_df.close)

In [ ]:
min_idx, max_idx = min(sdaf.keys()), max(sdaf.keys())
sdaf_price = dollar_df.iloc[min_idx, max_idx+1]
sdaf_price['sdaf'] = [sdaf[k] for k in sorted(sdaf)]
plt.plot(sdaf_price.close)
plt.plot(sdaf_price.sdaf)